# Headers/Startup

In [ ]:
# IMPORTS
import logging
import traceback
from os import environ as os_environ
from sys import stdout

import dotenv

dotenv.load_dotenv()

from datetime import datetime, timedelta

import pandas as pd

# for verifying DB updates
from db_engines import\
    RPRT_DB, WH_CONN_STR, WH_DB as DB

tmstmp_fmt: str = r'%Y-%m-%d %H:%M:%S'
query_date_fmt: str = r'%Y-%m-%d'

import re
from pathlib import Path
from threading import Thread

import pandas as pd
from pandas import DataFrame as Df


In [ ]:
# OTHER CONSTANTS
TODAY: str = datetime.now().strftime(query_date_fmt)
REPOS_PATH = Path(os_environ['PRMDIA_EVAN_LOCAL_LAKEPATH'])

XTRA_SQL_FILE = Path('billables_views.sql').name
PSQL_CMD: str = f"psql --file={XTRA_SQL_FILE} {WH_CONN_STR}"


In [ ]:
# LOGGING SETUP
LOG_FMT_DATE_STRM = r'%y%m%d|%H%M'
LOG_FMT_DATE_FILE = r'%Y-%m-%d %H:%M:%S'
LOG_FMT_FILE =\
    '%(asctime)s [%(name)s,%(funcName)s,%(module)s::%(levelname)s]>>%(message)s'
LOG_FMT_STRM =\
    '\x1b[32m%(asctime)s[%(name)s %(levelname)s]\x1b[0m >> %(message)s'

LOGGER = logging.getLogger(os_environ['PRMDIA_MM_LOGNAME'])
hdlr = logging.StreamHandler(stdout)
hdlr.setFormatter(
    logging.Formatter(
        fmt=LOG_FMT_STRM, datefmt=LOG_FMT_DATE_STRM))
# hdlr.setLevel(logging.DEBUG)
LOGGER.addHandler(hdlr)
LOGGER.setLevel(logging.INFO)

In [ ]:
# LOAD PM PHONE NUMBERS
pm_phone: Df
with RPRT_DB.connect() as conn:
    pm_phone = pd.read_sql_query(
        sql="""--sql
            SELECT phone_dir FROM dim_phone;
        """.replace('--sql\n', ''),
        con=conn
    )
pm_phones: list[int] = list(pm_phone['phone_dir'])

# create string for query
# print(*[i for i in pm_phone], sep=', ')
ph_not_in: str = ', '.join([str(i) for i in pm_phones])

log_msg = ', '.join([str(i) for i in pm_phones])
LOGGER.debug(f"PM phone nums excluded: \n{log_msg}")
del log_msg


In [ ]:
# check for active connections, else raise exception and bail
from db_engines import MySQL_OpErr, check_connection

for d in DB, RPRT_DB:
    try:
        check_connection(d)
    except MySQL_OpErr:
        raise Exception(f"\x1b[91mSEE BELOW/ABOVE\x1b[0m\n")
    else:
        pass

del MySQL_OpErr, check_connection


In [ ]:
# DATA LAKE VINTAGE CHECK
from table_config import AF_CFGS, ATT_FILE_CFG
rng = 5

ANSI = '\x1b[{clr}m'
GOOD_ANSI = '93'
BAD_ANSI = '1;91'
good_ansi = ANSI.format(clr=GOOD_ANSI)
bad_ansi = ANSI.format(clr=BAD_ANSI)
ANSI_RST = '\x1b[0m'
rpo_chk_prstr = (
    "❇️{an}{nm}{anr}, source or top of glob for ({ds}) "
    + "Repos Vintage: {an}{ts}{anr}"
)


AF_GLOB: str = AF_CFGS['src_label']
ATT_GLOB: str = ATT_FILE_CFG['src_label']
del AF_CFGS, ATT_FILE_CFG

# get recent mtimes
af_files: list[Path]
att_files: list[Path]
af_files, att_files = (
    list(REPOS_PATH.rglob(glob))
    for glob in (AF_GLOB, ATT_GLOB)
)

for l, t in ((af_files, 'af_message_data'), (att_files, 'att_data')):
    l.sort(reverse=True, key=lambda p: p.stat().st_mtime)

    for i in range(rng):
        p = l[i]
        t = datetime.fromtimestamp(p.stat().st_mtime)
        ts = t.strftime(tmstmp_fmt)
        nm = l[i].name

        now, dlt = datetime.now(), timedelta(hours=(16))
        clr = good_ansi if (now - t < dlt) else bad_ansi
        del now, dlt

        LOGGER.info(rpo_chk_prstr.format(
            an=clr, anr=ANSI_RST, nm=nm, ds=t, ts=ts))

        del clr, nm, ts

del ANSI, GOOD_ANSI, BAD_ANSI, ANSI_RST, AF_GLOB, ATT_GLOB


# ETL Scripts

In [ ]:
# ETL FROM REPOS
from etl_att_repos import main as att
from etl_af_repos import main as af
from etl_client_key import main as client
from etl_f_in_house_leads import main as inhouse

att_thr = Thread(target=att)
af_thr = Thread(target=af)
client_thr = Thread(target=client)
inhouse_thr = Thread(target=inhouse)
threads = (
    af_thr,
    att_thr,
    client_thr,
    inhouse_thr,
)

for t in threads:
    t.start()

for t in threads:
    t.join()

del att_thr, af_thr, threads, att, af

In [ ]:
# CREATE VIEW OF MASTER JOIN
LOGGER.info(f"Re-instating master join view.")
!{PSQL_CMD} 